In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import os

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from src.utils.time_series_procs import split_sequences
from src.models.transformer_ts_tf import Transformer

In [2]:
def get_times_series_data(df, cut_date):
    columns = list(df.columns.drop(["R28D"]))
    columns.append("R28D")
    df = df.reindex(columns=columns)
    df["Data/Hora"] = pd.to_datetime(df["Data/Hora"])
    df_train = df[df["Data/Hora"] < cut_date]
    df_test = df[df["Data/Hora"] >= cut_date]
    df_train = df_train.drop(["Data/Hora"], axis=1)
    df_test = df_test.drop(["Data/Hora"], axis=1)
    x_train, y_train = split_sequences(df_train.values, TIME_STEPS)
    x_test, y_test = split_sequences(df_test.values, TIME_STEPS)
    return x_train, y_train, x_test, y_test

In [3]:
class StandardScaler3DShape:
    def __init__(self):
        self.scaler = StandardScaler()

    def fit_transform(self, X, y=None):
        X_new = self.scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

    def fit(self, X, y=None):
        X_new = X.reshape(-1, X.shape[-1])
        self.scaler.fit(X_new)
        return self

    def transform(self, X, y=None):
        X_new = self.scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

In [4]:
def get_baseline_model(
    num_features, time_steps, num_heads=2, num_layers_enc=1, num_feed_forward=32
):
    model = Transformer(
        num_hid=num_features,
        time_steps=time_steps,
        num_head=num_heads,
        num_layers_enc=num_layers_enc,
        num_feed_forward=num_feed_forward,
    )

    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(
        optimizer=opt,
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [5]:
def make_repeated_time_series_k_fold(
    x,
    y,
    train_period,
    test_period,
    repeats=10,
    show_individual_results=True,
    show_final_result=True,
):
    results = []
    max_samples = x.shape[0]

    for _ in range(repeats):
        i = 0
        scores = []
        for _ in range(0, max_samples // train_period):
            x_train = x[i : i + train_period]
            y_train = y[i : i + train_period]
            x_test = x[i + train_period : i + train_period + test_period]
            y_test = y[i + train_period : i + train_period + test_period]
            i += train_period
            x_train = scaler.fit_transform(
                x_train.reshape(-1, x_train.shape[-1])
            ).reshape(x_train.shape)
            x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(
                x_test.shape
            )
            model = get_baseline_model(
                num_features=x_train.shape[-1], time_steps=x_train.shape[-2]
            )
            model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=0)
            rmse = model.evaluate(x_test, y_test, verbose=0)
            scores.append(rmse[1])
        results.append(scores)
        if show_individual_results:
            print("RMSE: %.3f (%.3f)" % (np.mean(scores), np.std(scores)))
    if show_final_result:
        print("\nRMSE: %.3f (%.3f)" % (np.mean(results), np.std(results)))
    return results

In [6]:
def make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats=10, timesteps_list=[3], show_results=True
):
    results = {}

    for timesteps in timesteps_list:
        x, y = split_sequences(df.values, timesteps)
        scores = make_repeated_time_series_k_fold(
            x,
            y,
            train_period,
            test_period,
            repeats=10,
            show_individual_results=False,
            show_final_result=False,
        )
        results[timesteps] = scores
        if show_results:
            print(
                "TIMESTEPS: %d RMSE: %.3f (%.3f)"
                % (timesteps, np.mean(scores), np.std(scores))
            )
    return results

<h3>Disable Tensorflow Warnings</h3>

In [7]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# FFT Transform

This notebook contains that experiments with the data transformed with fast fourier transform.

In [8]:
df_r3d_and_r7d = pd.read_csv(
    "../../../../data/interim/"
    + "r3d_and_r7d-no-resampling-with-fillna-and-fft-vars-only-"
    + "dados-historicos-partner_i-cement-CPIIF40.csv"
)
df_r3d_only = pd.read_csv(
    "../../../../data/interim/"
    + "r3d_only-no-resampling-with-fillna-and-fft-vars-only-"
    + "dados-historicos-partner_i-cement-CPIIF40.csv"
)
df_no_r3d_r7d = pd.read_csv(
    "../../../../data/interim/"
    + "no-r3d-r7d-no-resampling-with-fillna-and-fft-vars-only-"
    + "dados-historicos-partner_i-cement-CPIIF40.csv"
)

In [9]:
seed = 47
scaler = StandardScaler()

timesteps_list = [1, 3, 5, 7, 10, 15, 20]
repeats = 10

# Transformer Encoder

# Configs:

1. TIMESTEPS: [1, 3, 5, 7, 10, 15, 20]

2. Epochs: 100

3. Batch size: 32

4. num heads: 2

5. FF Layers enc: 1

      5.1 Units 32

6. Optimization Algorithm: Adam
    
    6.1. Learning Rate: 1e-2

# Times Series Repeated KFold Cross validation - different Timesteps values

Here we will make a Times Series KFold Cross validation with all 3 datasets. Here we are using the Blocking Time Series Split method. And with timeseries Split method. We also do it with different number of timesteps


<h3> Time Series Repeated KFold Cross Validation with vary timesteps - df_no_r3d_r7d</h3>

<b>Dataset: df_no_r3d_r7d:</b> In this dataset the R3D and R7D variables are not considered.

In [10]:
df = df_no_r3d_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora'], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [11]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [12]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.118 (0.386)
TIMESTEPS: 3 RMSE: 2.161 (0.247)
TIMESTEPS: 5 RMSE: 2.060 (0.312)
TIMESTEPS: 7 RMSE: 2.124 (0.534)
TIMESTEPS: 10 RMSE: 2.338 (0.466)
TIMESTEPS: 15 RMSE: 2.837 (0.578)
TIMESTEPS: 20 RMSE: 3.030 (1.122)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [13]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [14]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.972 (0.278)
TIMESTEPS: 3 RMSE: 2.207 (0.354)
TIMESTEPS: 5 RMSE: 2.213 (0.427)
TIMESTEPS: 7 RMSE: 2.088 (0.511)
TIMESTEPS: 10 RMSE: 2.781 (0.298)
TIMESTEPS: 15 RMSE: 2.206 (0.062)
TIMESTEPS: 20 RMSE: 2.640 (0.169)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [15]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [16]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 3.331 (0.337)
TIMESTEPS: 3 RMSE: 3.423 (0.476)
TIMESTEPS: 5 RMSE: 3.347 (0.483)
TIMESTEPS: 7 RMSE: 3.649 (0.416)
TIMESTEPS: 10 RMSE: 3.186 (0.479)
TIMESTEPS: 15 RMSE: 3.064 (0.404)
TIMESTEPS: 20 RMSE: 2.778 (0.364)


<h3> Time Series Repeated KFold Cross Validation - df_r3d_only</h3>

<b>Dataset: df_r3d_only:</b> In this dataset only R3D variable is considered.

In [17]:
df = df_r3d_only.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora'], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [18]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [19]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.119 (0.506)
TIMESTEPS: 3 RMSE: 2.358 (0.448)
TIMESTEPS: 5 RMSE: 2.410 (0.412)
TIMESTEPS: 7 RMSE: 2.611 (0.538)
TIMESTEPS: 10 RMSE: 2.066 (0.555)
TIMESTEPS: 15 RMSE: 1.779 (0.427)
TIMESTEPS: 20 RMSE: 2.147 (0.447)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [20]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [21]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 3.479 (0.361)
TIMESTEPS: 3 RMSE: 3.221 (0.467)
TIMESTEPS: 5 RMSE: 3.406 (0.379)
TIMESTEPS: 7 RMSE: 3.067 (0.304)
TIMESTEPS: 10 RMSE: 3.500 (0.406)
TIMESTEPS: 15 RMSE: 2.862 (0.692)
TIMESTEPS: 20 RMSE: 2.395 (1.675)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [22]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [23]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.603 (0.288)
TIMESTEPS: 3 RMSE: 2.734 (0.334)
TIMESTEPS: 5 RMSE: 2.578 (0.323)
TIMESTEPS: 7 RMSE: 2.250 (0.522)
TIMESTEPS: 10 RMSE: 2.170 (0.395)
TIMESTEPS: 15 RMSE: 2.687 (0.609)
TIMESTEPS: 20 RMSE: 2.829 (0.477)


<h3> Time Series Repeated KFold Cross Validation - df_r3d_and_r7d</h3>

<b>Dataset: df_r3d_and_r7d:</b> In this dataset both R3D and R7D variables are considered.

In [24]:
df = df_r3d_and_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora'], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [25]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [26]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.104 (0.348)
TIMESTEPS: 3 RMSE: 2.436 (0.532)
TIMESTEPS: 5 RMSE: 2.501 (0.548)
TIMESTEPS: 7 RMSE: 2.330 (0.644)
TIMESTEPS: 10 RMSE: 2.252 (0.289)
TIMESTEPS: 15 RMSE: 2.209 (1.348)
TIMESTEPS: 20 RMSE: 2.195 (0.428)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [27]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [28]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 3.377 (0.377)
TIMESTEPS: 3 RMSE: 3.092 (0.191)
TIMESTEPS: 5 RMSE: 2.930 (0.459)
TIMESTEPS: 7 RMSE: 3.320 (0.343)
TIMESTEPS: 10 RMSE: 3.585 (0.266)
TIMESTEPS: 15 RMSE: 2.892 (0.813)
TIMESTEPS: 20 RMSE: 1.868 (0.688)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [29]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [30]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.744 (0.294)
TIMESTEPS: 3 RMSE: 2.506 (0.564)
TIMESTEPS: 5 RMSE: 2.516 (0.492)
TIMESTEPS: 7 RMSE: 2.513 (0.333)
TIMESTEPS: 10 RMSE: 2.580 (0.464)
TIMESTEPS: 15 RMSE: 2.366 (0.578)
TIMESTEPS: 20 RMSE: 3.184 (0.306)
